In [ ]:
#| default_exp data

In [ ]:
#| export
import numpy as np
import tensorflow as tf

In [ ]:
#| hide
tf.config.set_visible_devices([], device_type='GPU')

In [ ]:
#| exporti
def load_cifar10():
    from tensorflow.keras.datasets import cifar10
    
    (X_train, Y_train), (X_test, Y_test) = cifar10.load_data()
    X_train = X_train/255.
    X_test = X_test/255.
    Y_train = Y_train.astype(np.int32)[...,0]
    Y_test = Y_test.astype(np.int32)[...,0]
    
    return tf.data.Dataset.from_tensor_slices((X_train, Y_train)), tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
#| exporti
def load_mnist():
    from tensorflow.keras.datasets import mnist
    
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    X_train = X_train[...,None]/255.
    X_test = X_test[...,None]/255.
    Y_train = Y_train.astype(np.int32)
    Y_test = Y_test.astype(np.int32)

    return tf.data.Dataset.from_tensor_slices((X_train, Y_train)), tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [ ]:
#| exporti
def load_cats_vs_dogs():
    import tensorflow_datasets as tfds

    dst = tfds.load("cats_vs_dogs", split="train")
    dst = dst.map(lambda x: (x["image"], x["label"]), num_parallel_calls=tf.data.AUTOTUNE)
    return dst, None

In [ ]:
#| export
def load_data(name: str):
    if name == "cifar10":
        dst_train, dst_val = load_cifar10()

    elif name == "mnist":
        dst_train, dst_val = load_mnist()

    elif name == "cats_vs_dogs":
        dst_train, dst_val = load_cats_vs_dogs()

In [ ]:
for name in ["mnist", "cifar10", "cats_vs_dogs"]:
    load_data(name)